In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from random import randint
from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
    transforms.RandomRotation(randint(1,360)),
    transforms.RandomHorizontalFlip(),

])

In [4]:
train_path = "../Remote sensing paper/Final Dataset/train"
test_path = "../Remote sensing paper/Final Dataset/test"
val_path = "../Remote sensing paper/Final Dataset/val"

In [5]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [6]:
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=transforms,is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path,transform=transforms,is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path,transform=transforms,is_valid_file=check_image)

In [7]:
batch_size = 64
num_classes = 10

In [8]:
train_loader = DataLoader(dataset=train_data,batch_size=batch_size)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
val_loader = DataLoader(dataset=val_data,batch_size=batch_size)

In [9]:
if torch.cuda.is_available:
    device = torch.device('cuda')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

In [10]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=2, aux_avai=False, device=device):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            if aux_avai:
                loss = loss_fn(output, targets) + 0.3*loss_fn(aux1, targets) + 0.3*loss_fn(aux2, targets)
            else:
                loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output,_,_ = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [18]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
model.fc = nn.Linear(1024, num_classes)
model.to(device)

Using cache found in C:\Users\manje/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [21]:
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.001,weight_decay=1e-4)

loss = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=20, gamma=0.1, last_epoch=-1,verbose=True)

c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
train(model=model,optimizer=optimizer,loss_fn=loss,train_loader=train_loader,val_loader=val_loader,epochs=100,device=device)